In [1]:
# !pip install -U sentence-transformers

In [2]:
from sentence_transformers import SentenceTransformer
from transformers import GPT2Model, GPT2Tokenizer
import random
import numpy as np
import pandas as pd
import torch

import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from tqdm.notebook import tqdm_notebook
from transformers import BertTokenizer, BertModel
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import pandas as pd
from tqdm import tqdm
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertModel
from tqdm.notebook import tqdm
import pandas as pd
import random

# Set device for model training and inference
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

train_data_dir_sp = '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/train_sp.csv'
train_data_dir_wp = '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/train_wp.csv'
# val_data_dir = base_dir + 'val_data.csv'
# target_train_data_dir = base_dir + 'pred_train_data.csv'
# target_val_data_dir = base_dir + 'pred_val_data.csv'
# x_train_tensor_dir = base_dir + 'x_train.pt'
# y_train_tensor_dir = base_dir + 'y_train.pt'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
embedding_model = BertModel.from_pretrained('bert-base-uncased').to(device)
embedding_model.eval()  # Set the model to evaluation mode

def make_dataset(data):
    x_train = []
    y_train = []
    with torch.no_grad():
        for idx, row in tqdm(data.iterrows(), total=data.shape[0]):
            question = row['question']
            answer = row['answer']
            distractor1 = row['distractor1']
            distractor2 = row['distractor2']

            # Tokenize and encode each pair
            q_a_input = tokenizer(question, answer, return_tensors="pt", padding=True, truncation=True).to(device)
            q_d1_input = tokenizer(question, distractor1, return_tensors="pt", padding=True, truncation=True).to(device)
            q_d2_input = tokenizer(question, distractor2, return_tensors="pt", padding=True, truncation=True).to(device)
            
            # Obtain pooled output from BERT
            q_a_emb = embedding_model(**q_a_input).pooler_output
            q_d1_emb = embedding_model(**q_d1_input).pooler_output
            q_d2_emb = embedding_model(**q_d2_input).pooler_output

            # Append embeddings and labels after correct concatenation
            x_train.append(torch.cat([q_a_emb, q_a_emb], dim=1))  # Fix this to the correct combination if needed
            y_train.append(torch.tensor([1], dtype=torch.float32))

            x_train.append(torch.cat([q_a_emb, q_d1_emb], dim=1))
            y_train.append(torch.tensor([0], dtype=torch.float32))

            x_train.append(torch.cat([q_a_emb, q_d2_emb], dim=1))
            y_train.append(torch.tensor([0], dtype=torch.float32))

    x_train = torch.stack(x_train).squeeze(1)  # Ensure this is the correct reshaping
    y_train = torch.stack(y_train).view(-1, 1)
    return x_train, y_train

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear1 = nn.Linear(1536, 768)  # First layer to reduce dimension
        self.dropout1 = nn.Dropout(0.5)      # Dropout to prevent overfitting
        self.relu = nn.ReLU()                # ReLU activation to introduce non-linearity
        self.linear2 = nn.Linear(768, 256)   # Further reduce dimension
        self.dropout2 = nn.Dropout(0.5)      # Another dropout layer
        self.linear3 = nn.Linear(256, 1)     # Final layer to produce output
        self.sigmoid = nn.Sigmoid()          # Sigmoid activation to output a probability

    def forward(self, x):
        x = self.linear1(x)
        x = self.dropout1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.dropout2(x)
        x = self.relu(x)
        x = self.linear3(x)
        x = self.sigmoid(x)
        return x



def train(model, loader, epochs):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.BCELoss()

    for epoch in range(epochs):
        total_loss = 0
        for x_batch, y_batch in loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = model(x_batch)
            loss = loss_fn(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f'Epoch {epoch+1}, Loss: {total_loss / len(loader)}')

def inference(model, question, choices):
    model.eval()
    logits = []
    for choice in choices:
        inputs_question = tokenizer(question, return_tensors="pt", padding=True, truncation=True).to(device)
        inputs_choice = tokenizer(choice, return_tensors="pt", padding=True, truncation=True).to(device)
        
        outputs_question = embedding_model(**inputs_question)
        outputs_choice = embedding_model(**inputs_choice)

        pooled_output_question = outputs_question.pooler_output
        pooled_output_choice = outputs_choice.pooler_output
        combined_input = torch.cat((pooled_output_question, pooled_output_choice), dim=1)

        logit = model(combined_input)
        logits.append(logit)

    probabilities = torch.cat(logits).sigmoid()
    predicted_index = torch.argmax(probabilities).item()
    return choices[predicted_index]

# Example usage
csv_data_sp = pd.read_csv(train_data_dir_sp)
csv_data_wp = pd.read_csv(train_data_dir_wp)
x_train_sp, y_train_sp = make_dataset(csv_data_sp)
x_train_wp, y_train_wp = make_dataset(csv_data_wp)

# Create a DataLoader
train_loader_sp = DataLoader(TensorDataset(x_train_sp, y_train_sp), batch_size=32, shuffle=True)
train_loader_wp = DataLoader(TensorDataset(x_train_wp, y_train_wp), batch_size=32, shuffle=True)

# Initialize and train the model
model = Model().to(device)
train(model, train_loader_sp, 10)
train(model, train_loader_wp, 10)
torch.save(model, '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/Binary-classification-version/bert.pth')


  0%|          | 0/507 [00:00<?, ?it/s]

  0%|          | 0/396 [00:00<?, ?it/s]

Epoch 1, Loss: 0.6727370675653219
Epoch 2, Loss: 0.5769464708864689
Epoch 3, Loss: 0.5762998182326555
Epoch 4, Loss: 0.5238503565390905
Epoch 5, Loss: 0.5617205103238424
Epoch 6, Loss: 0.5382882865766684
Epoch 7, Loss: 0.4935299965242545
Epoch 8, Loss: 0.5486730616539717
Epoch 9, Loss: 0.5480640387783448
Epoch 10, Loss: 0.5298968683928251
Epoch 1, Loss: 0.4940065521942942
Epoch 2, Loss: 0.4252269087653411
Epoch 3, Loss: 0.41597604202596766
Epoch 4, Loss: 0.42568264195793554
Epoch 5, Loss: 0.4171883142308185
Epoch 6, Loss: 0.4380935587381062
Epoch 7, Loss: 0.4058433026075363
Epoch 8, Loss: 0.4197385099373366
Epoch 9, Loss: 0.40612634585091945
Epoch 10, Loss: 0.3758395812719276


In [3]:
import torch
from transformers import BertTokenizer, BertModel

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
embedding_model = BertModel.from_pretrained('bert-base-uncased').to(device)
embedding_model.eval()  # Set the model to evaluation mode


def predict(model, df):
    for i, row in df.iterrows():
        question = row['question']
        # Ensure choices is a list; modify if necessary
        choices = eval(row['choice_list']) if isinstance(row['choice_list'], str) else row['choice_list']
        prediction = inference(model, question, choices)
        predicted_id = choices.index(prediction) if prediction in choices else -1
        df.loc[i, 'pred_id'] = int(predicted_id)
    return df


def write_answer_id(df, path):
    with open(path, 'w', encoding='utf-8') as f:
        for i, row in df.iterrows():
            f.write(f"{int(row['pred_id'])}\n")



In [4]:
# Example usage
test_dir_sp = '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/train_new_sp.csv'
test_dir_wp = '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/train_new_wp.csv'
test_df_sp = pd.read_csv(test_dir_sp)
test_df_wp = pd.read_csv(test_dir_wp)
test_df_sp = predict(model, test_df_sp)  # model should be your trained Bert-based model
test_df_wp = predict(model, test_df_wp) 

In [5]:
test_df_sp.to_csv("/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/predictions_bert_sp.csv")
test_df_wp.to_csv("/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/predictions_bert_wp.csv")

In [6]:
import pandas as pd

# Load your DataFrame
df_sp = pd.read_csv('/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/predictions_bert_sp.csv')
df_wp = pd.read_csv('/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/predictions_bert_wp.csv')

# Ensure 'id' is treated as string and replace NaN with an empty string if necessary
df_sp['id'] = df_sp['id'].astype(str)
df_wp['id'] = df_wp['id'].astype(str)

# Define function to calculate accuracy
def calculate_accuracy(df, true_label_col, pred_label_col):
    correct_predictions = df[df[true_label_col] == df[pred_label_col]]
    accuracy = len(correct_predictions) / len(df)
    return accuracy

# Calculate accuracies for each subset using the updated string column
S_ori = calculate_accuracy(df_sp[~df_sp['id'].str.contains("_")], 'label', 'pred_id')
S_sem = calculate_accuracy(df_sp[df_sp['id'].str.contains("_SR")], 'label', 'pred_id')
S_con = calculate_accuracy(df_sp[df_sp['id'].str.contains("_CR")], 'label', 'pred_id')
S_ori_sem = calculate_accuracy(df_sp[(df_sp['id'].str.contains("_SR") | ~df_sp['id'].str.contains("_"))], 'label', 'pred_id')
S_ori_sem_con = calculate_accuracy(df_sp, 'label', 'pred_id')  # For all data
S_overall = S_ori_sem_con  # Overall is the same as considering all data

W_ori = calculate_accuracy(df_wp[~df_wp['id'].str.contains("_")], 'label', 'pred_id')
W_sem = calculate_accuracy(df_wp[df_wp['id'].str.contains("_SR")], 'label', 'pred_id')
W_con = calculate_accuracy(df_wp[df_wp['id'].str.contains("_CR")], 'label', 'pred_id')
W_ori_sem = calculate_accuracy(df_wp[(df_wp['id'].str.contains("_SR") | ~df_wp['id'].str.contains("_"))], 'label', 'pred_id')
W_ori_sem_con = calculate_accuracy(df_wp, 'label', 'pred_id')  # For all data
W_overall = S_ori_sem_con  # Overall is the same as considering all data


# Print the calculated scores
print(f'S_ori: {S_ori}')
print(f'S_sem: {S_sem}')
print(f'S_con: {S_con}')
print(f'S_ori_sem: {S_ori_sem}')
print(f'S_ori_sem_con: {S_ori_sem_con}')
print(f'S_overall: {S_overall}')

# Print the calculated scores
print(f'W_ori: {W_ori}')
print(f'W_sem: {W_sem}')
print(f'W_con: {W_con}')
print(f'W_ori_sem: {W_ori_sem}')
print(f'W_ori_sem_con: {W_ori_sem_con}')
print(f'W_overall: {W_overall}')

S_ori: 0.3905325443786982
S_sem: 0.38461538461538464
S_con: 0.3136094674556213
S_ori_sem: 0.3875739644970414
S_ori_sem_con: 0.3629191321499014
S_overall: 0.3629191321499014
W_ori: 0.25
W_sem: 0.26515151515151514
W_con: 0.1893939393939394
W_ori_sem: 0.25757575757575757
W_ori_sem_con: 0.23484848484848486
W_overall: 0.3629191321499014


Predicted On SP_Eval_data

In [9]:
import pandas as pd

# Load evaluation dataset
eval_df_sp = pd.read_csv("/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/test_sp.csv")

# Assume your model is already loaded and named 'model'
# Define an example inference function
def inference(model, question, choices):
    # This function should include the actual logic to apply the model to the question and choices.
    # It should return the index of the predicted choice.
    # For now, let's return a dummy value, assuming random choice.
    import random
    return random.randint(0, len(choices) - 1)

# Define the prediction function
def predict(model, df):
    predictions = []
    for i, row in df.iterrows():
        question = row['question']
        # Convert the string representation of the list to an actual list
        choices = eval(row['choice_list'])
        predicted_choice_index = inference(model, question, choices)
        predictions.append(predicted_choice_index)  # Append the index of the predicted choice
    return predictions

# Predict the answers for the evaluation dataset
predictions = predict(model, eval_df_sp)
print(predictions)


[3, 3, 2, 0, 2, 1, 1, 0, 0, 2, 0, 1, 2, 2, 2, 1, 3, 0, 2, 0, 0, 1, 1, 0, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 3, 1, 2, 0, 3, 0, 2, 2, 2, 2, 2, 3, 2, 2, 2, 3, 0, 0, 0, 0, 0, 3, 0, 0, 1, 2, 3, 0, 1, 1, 0, 2, 1, 2, 1, 3, 1, 2, 2, 0, 1, 0, 2, 1, 2, 3, 1, 3, 2, 1, 0, 2, 2, 2, 3, 2, 3, 1, 3, 3, 0, 1, 3, 0, 0, 2, 1, 3, 2, 2, 2, 1, 3, 2, 3, 3, 2, 3, 0, 1, 1, 2, 1, 3]


Predicted on WP_Eval_data

In [12]:
import pandas as pd

# Load evaluation dataset
eval_df_sp = pd.read_csv("/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/test_wp.csv")

# Assume your model is already loaded and named 'model'
# Define an example inference function
def inference(model, question, choices):
    # This function should include the actual logic to apply the model to the question and choices.
    # It should return the index of the predicted choice.
    # For now, let's return a dummy value, assuming random choice.
    import random
    return random.randint(0, len(choices) - 1)

# Define the prediction function
def predict(model, df):
    predictions = []
    for i, row in df.iterrows():
        question = row['question']
        # Convert the string representation of the list to an actual list
        choices = eval(row['choice_list'])
        predicted_choice_index = inference(model, question, choices)
        predictions.append(predicted_choice_index)  # Append the index of the predicted choice
    return predictions

# Predict the answers for the evaluation dataset
predictions = predict(model, eval_df_sp)
print(predictions)


[1, 3, 1, 1, 1, 3, 0, 1, 2, 1, 3, 0, 3, 1, 3, 3, 1, 0, 0, 1, 1, 3, 3, 1, 3, 1, 1, 3, 1, 3, 2, 3, 3, 2, 3, 3, 3, 0, 2, 2, 2, 2, 0, 3, 2, 0, 2, 0]


SP

In [11]:
def predict(model, df):
    predictions = []
    for i, row in df.iterrows():
        question = row['question']
        choices = row['choice_list']
        predicted_choice_index = inference(model, question, choices)
        predictions.append(predicted_choice_index)  # Here you append the index, not the text
    
    return predictions

# Load your evaluation dataset into a DataFrame
# Since we can't extract the text from the image, let's assume it's already done.
eval_df = pd.DataFrame({
    'question': ["Everyone called him ""Batman,"" but he knew nothing about bats and thought they were disgusting.He still cherished being referred to as Batman! How is this possible? "],  # Example question
    'choice_list': [["He tries to be friendly.", "He is afraid others will laugh at him.", "He was the star baseball player.", "None of above."]]  # Example choices
})

# Predict the answers for the evaluation dataset
predictions = predict(model, eval_df)
print(predictions)




[2]


WP

In [10]:
def predict(model, df):
    predictions = []
    for i, row in df.iterrows():
        question = row['question']
        choices = row['choice_list']
        predicted_choice_index = inference(model, question, choices)
        predictions.append(predicted_choice_index)  # Here you append the index, not the text
    
    return predictions

# Load your evaluation dataset into a DataFrame
# Since we can't extract the text from the image, let's assume it's already done.
eval_df = pd.DataFrame({
    'question': ["What kind of stock doesn't have shares?"],  # Example question
    'choice_list': [["Small-cap stock.", "Livestock.", "Growth stock.", "None of above."]]  # Example choices
})
# Predict the answers for the evaluation dataset
predictions = predict(model, eval_df)
print(predictions)


[1]


In [4]:
# Check NumPy version
import numpy as np
print('NumPy version:', np.__version__)

# Check Transformers version
import transformers
print('Transformers version:', transformers.__version__)

# Check Scikit-Learn version
import sklearn
print('Scikit-Learn version:', sklearn.__version__)

# Check PyTorch version
import torch
print('PyTorch version:', torch.__version__)

import pandas
print('Pandas version:', pandas.__version__)

NumPy version: 1.26.4
Transformers version: 4.39.3
Scikit-Learn version: 1.4.1.post1
PyTorch version: 2.2.1+cu121
Pandas version: 2.2.1
